Imports

In [1]:
import csv
import pandas as pd
from datetime import datetime
from datetime import timedelta
import os
#from subprocess import call
import subprocess

Functions

In [2]:
#convert .FIT to .csv with the help of FIT_sdk
def convertFITToCsv(file):
    source_file = "/home/julio/strides/data/fit_raw/"+file+".FIT"
    target_file = "/home/julio/strides/data/fit2csv/"+file+".csv"
    #call(["java", "-jar", "/home/julio/strides/fit_sdk/java/FitCSVTool.jar", "-b", source_file, target_file])
    subprocess.run(["java", "-jar", "/home/julio/strides/fit_sdk/java/FitCSVTool.jar", "-b", source_file, target_file], stdout=subprocess.PIPE)

In [3]:
def secondsToUTC (dataset):
    #1989-12-31 00:00:00
    zero = datetime(1989,12,31,0,0,0)
    #print (datetime.strftime(zero,"%b %d %Y %H:%M:%S"))
    new_dataset = []
    for d in dataset:
        n = str(zero + timedelta(seconds=d))
        new_dataset.append(n)
    return new_dataset

In [4]:
def prepareCSV (filepath, file, filetype):
    print(file)
    #Read csv
    df = pd.read_csv(filepath+file+filetype)
    
    #filter for local number == 7 and type == Date
    df_filtered = df.loc[df['Local Number'] == 7]
    df_filtered = df_filtered.loc[df_filtered['Type'] == 'Data']
    
    #convert position and time
    df_filtered['Value 2'] = df_filtered['Value 2']* 180/(2**31)
    df_filtered['Units 2'] = "degrees"

    df_filtered['Value 3'] = df_filtered['Value 3']* 180/(2**31)
    df_filtered['Units 3'] = "degrees"

    df_filtered['Value 1'] = secondsToUTC(df_filtered['Value 1'])
    df_filtered['Units 1'] = "UTC"
    
    # drop unnecessary data
    toBeDropped = []
    counter = 10
    field_str = 'Field '
    value_str = 'Value '
    units_str = 'Units '

    upperBound = (len(df.columns) - 31)/3 + 10
    while counter < upperBound:
        toBeDropped.append(field_str+str(counter))
        toBeDropped.append(value_str+str(counter))
        toBeDropped.append(units_str+str(counter))
        counter = counter + 1

    toBeDropped.append("Type")
    toBeDropped.append("Local Number")
    toBeDropped.append("Message")
    toBeDropped.append("Unnamed: "+str(len(df.columns)-1))
    
    df_filtered = df_filtered.drop(columns=toBeDropped)
    
    # write csv
    df_filtered.to_csv(filepath+file+'_prepared'+filetype)

In [5]:
#Get file names in directory
def getFiles(filepath, filetype):
    files = []  
    for file in os.listdir(filepath):
        if file.endswith(filetype) and not file.endswith("prepared"+filetype):
            files.append(file.split('.')[0])
    return files

Convert FIT to csv

In [6]:
files = getFiles(filepath='/home/julio/strides/data/fit_raw/', filetype='.FIT')
#print (files)
for file in files:
    convertFITToCsv(file)

Prepare csv files

In [7]:
files = getFiles(filepath='/home/julio/strides/data/fit2csv/', filetype='.csv')
#print (files)
for file in files:
    prepareCSV (filepath='/home/julio/strides/data/fit2csv/', file=file, filetype='.csv')

913F2746
91DD1310
91QA1309
91H73828
91HK2904
91QA1841
91SI5602
916A2805
911E0950
911D3633
91KE0003
91VI5920
91BE1243
91LI0343
914H2817
921B4648
91RB2029


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (35,62,65,69,71) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


91NJ1712
8CUG1206
91CG2815
91KB2254
91U70725
91VI2925
91AH1749
915F0120
911B3000
919H2158
91BC4602
911C4903
91NK5221
91JE0641
91FJ5751
